In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

(mr0_pypulseq_example)=
# Simulate pypulseq example files
here the pypulseq example files from pypulseq/seq_examples.scripts

In [ ]:
#@title 1.a choose pypulseq example seq files (first call generates them)
import numpy as np
# newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float
np.complex = complex

# Only build some for faster doc build, you can uncomment all
# from pypulseq.seq_examples.scripts import write_epi
# from pypulseq.seq_examples.scripts import write_epi_se
# from pypulseq.seq_examples.scripts import write_epi_se_rs
from pypulseq.seq_examples.scripts import write_gre
# from pypulseq.seq_examples.scripts import write_gre_label
# from pypulseq.seq_examples.scripts import write_haste
# from pypulseq.seq_examples.scripts import write_tse
# from pypulseq.seq_examples.scripts import write_ute

#@title choose pypulseq example
seq_file = "gre_pypulseq.seq" #@param ["epi_pypulseq.seq", "epi_se_pypulseq.seq", "epi_se_rs_pypulseq.seq", "gre_pypulseq.seq", "gre_label_pypulseq.seq", "haste_pypulseq.seq", "tse_pypulseq.seq", "ute_pypulseq.seq"] {allow-input: true}

In [ ]:
#@title 2. simulate for brain phantom
import MRzeroCore as mr0
import pypulseq as pp
import torch
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

seq = pp.Sequence()
seq.read(seq_file)

print('load phantom')
# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
sz = [64, 64]
obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
brain_phantom_res = 64 #@param {type:"slider", min:16, max:128, step:16}
obj_p = obj_p.interpolate(brain_phantom_res, brain_phantom_res, 1)
obj_p.B0[:] = 0
plot_phantom = True #@param {type:"boolean"}
if plot_phantom: obj_p.plot()

obj_p = obj_p.build()
print('simulate (2D) \n' + seq_file)
seq0 = mr0.Sequence.import_file(seq_file)
# seq0.plot_kspace_trajectory()

In [ ]:
# Simulate the sequence

graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p, print_progress=False)
#@title 3. Plot sequence and signal
sp_adc, t_adc = mr0.util.pulseq_plot(seq=seq,signal=signal.numpy())

# Unfortunately, we need to limit the resolution as reco_adjoint is very RAM-hungy
print('reconstruct and plot')
seq0.plot_kspace_trajectory()

reco = mr0.reco_adjoint(signal, seq0.get_kspace(), resolution=(64, 64, 1), FOV=(0.22, 0.22, 1))
plt.figure()
plt.subplot(121)
plt.title("Magnitude")
plt.imshow(reco[:, :, 0].T.abs(), origin="lower")
plt.colorbar()
plt.subplot(122)
plt.title("Phase")
plt.imshow(reco[:, :, 0].T.angle(), origin="lower", vmin=-np.pi, vmax=np.pi)
plt.colorbar()
plt.show()